# Importing the dependencies

In [76]:
import pandas as pd
import chromadb
import sqlite3

import google.generativeai as genai

import os
from dotenv import load_dotenv


load_dotenv()
GOOGLE_API_KEY_MOVIE_RECOMMENDER = os.getenv("GOOGLE_API_KEY_MOVIE_RECOMMENDER")
genai.configure(api_key=GOOGLE_API_KEY_MOVIE_RECOMMENDER)

# Chroma DB and Key Based Database Utilities

In [114]:
keyword_data_path = "../CHROMA_DATABASE/chroma.sqlite3"
client_key = chromadb.PersistentClient(path = keyword_data_path)
movie_collection = client_key.get_collection("MOVIES")

In [78]:
# Find similar movies based on the movie title from the Chroma database
def find_similar_movies(movie_title, data, movie_collection, top_k=6):
    try:
        movie_row = data[data['title'] == movie_title]
        
        if movie_row.empty:
            return f"Movie title '{movie_title}' not found in the database."
        

        query_embedding = eval(movie_row.iloc[0]['embeddings'])
        
        results = movie_collection.query(
            query_embeddings=[query_embedding],
            n_results=top_k
        )
        
        final_movies = []
        for movie in results['metadatas'][0]:
            final_movies.append(movie)
            
        return final_movies[1:]
        

    except Exception as e:
        print(f"Error occurred during request: {e}")
        return []

# SQL Database Inferencing Utilities

In [79]:
database = "../SQL_Database/Movies.db"
database_key_based = pd.read_sql_query("SELECT m.* FROM Movies_Key_Based AS m", sqlite3.connect(database))
database_query_based = pd.read_sql_query("SELECT m.* FROM Movies_Database AS m", sqlite3.connect(database))

In [113]:
data = pd.read_sql_query("SELECT m.* FROM Movies_Database AS m", sqlite3.connect(database))
data.head()

,id,IMDB_ID,title,release_year,genres,vote_average,cast,Director,keywords,reviews,review_sentiment,review_summary,poster_path,backdrop_path
0,283995,tt3896198,Guardians of the Galaxy Vol. 2,2017,"['Adventure', 'Action', 'Sci-Fi']",7.623,"['Chris Pratt', 'Zoe Saldaña', 'Dave Bautista'...",James Gunn,"['demi god', 'alien creature', 'sarcasm', 'cra...","[""Despite being a huge comic book nerd I was n...","['positive', 'positive', 'positive', 'positive...",Guardians of the Galaxy Vol. 2 elicited a gene...,/y4MBh0EjBlMuOzv9axM4qJlmhzz.jpg,/aJn9XeesqsrSLKcHfHP4u5985hn.jpg
1,480530,tt6343314,Creed II,2018,['Drama'],6.990,"['Michael B. Jordan', 'Sylvester Stallone', 'T...",Steven Caple Jr.,"['baby', 'training montage', 'sequel', 'boxing...","[""This movie is not as good as the first Creed...","['negative', 'positive', 'positive', 'positive...",Creed II elicits a mixed response from audienc...,/v3QyboWRoA4O9RbcsqH8tJMe8EB.jpg,/xTYGN1b3XkOtODryXTKgdXLtPMz.jpg
2,299536,tt4154756,Avengers: Infinity War,2018,"['Adventure', 'Action', 'Sci-Fi']",8.260,"['Robert Downey Jr.', 'Chris Hemsworth', 'Mark...",Anthony RussoJoe Russo,"['superhero', 'ensemble cast', 'marvel cinemat...","[""Avengers infinity war is an emotional roller...","['positive', 'positive', 'positive', 'positive...",Avengers: Infinity War elicits a generally pos...,/7WsyChQLEftFiDOVTGkv3hFpyyt.jpg,/mDfJG3LC3Dqb67AZ52x3Z0jU0uB.jpg
3,299534,tt4154796,Avengers: Endgame,2019,"['Adventure', 'Sci-Fi', 'Action']",8.268,"['Robert Downey Jr.', 'Chris Evans', 'Mark Ruf...",Anthony RussoJoe Russo,"['time travel', 'superhero', 'super villain', ...","[""But its a pretty good film. A bit of a mess ...","['positive', 'positive', 'positive', 'positive...",Avengers: Endgame elicited a largely positive ...,/or06FN3Dka5tukK1e9sl16pB3iy.jpg,/7RyHsO4yDXtBv1zUU3mTpHeQ0d5.jpg
4,337167,tt4477536,Fifty Shades Freed,2018,"['Drama', 'Romance']",6.699,"['Dakota Johnson', 'Jamie Dornan', 'Eric Johns...",James Foley,"['sex scene', 'wedding ceremony', 'bondage', '...","[""The first of the three that is actually emot...","['positive', 'negative', 'negative', 'negative...",The audience reaction to Fifty Shades Freed is...,/9ZedQHPQVveaIYmDSTazhT3y273.jpg,/9ywA15OAiwjSTvg3cBs9B7kOCBF.jpg


In [80]:
def process_sql_query(sql_command,db):
    try: 
        if(type(sql_command)==str):   
            
            con = sqlite3.connect(db)
            cur = con.cursor()
            cur.execute(sql_command)
            
            rows = cur.fetchall()
            
            column_names = [description[0] for description in cur.description]
            list_of_dicts = [dict(zip(column_names, row)) for row in rows]

            con.close()
            
            return list_of_dicts if type(list_of_dicts) == list else []
        
        elif type(sql_command)==list:
            return sql_command
        
    except Exception as e:
        print(f"Error occurred during request: {e}")
        return ""

# Prompt Generation and Knowledge Base Utilities

In [81]:
prompts = [
    """
    You are an expert at converting English text to SQL code. Here, the SQL Database you are getting is called Movies_Database and 
    has the following columns : \n\n
    
        1. id
        2. IMDB_ID
        3. title
        4. release_year
        5. genres
        6. vote_average
        7. cast
        8. Director
        9. keywords
        10. reviews
        11. review_sentiment
        12. review_summary
        13. poster_path
        14. backdrop_path
        
        
    \n\nNow, the above columns which I have mentioned are the columns of the table Movies_Database and are exactly the same as they are i.e. are case sensitive.
    So, make sure you do not make any mistakes in the column names.\n\n
    
    \n\nNow, if the text query you get talks about recommending movies based on release_year, vote_average and/or Director you need to convert 
    it into a sql command as shown in the example below. Since these are strings/numeric values, you have to convert it as it is.\n\n
    
        
    \n\nTake this for an example. If the text says, "Recommend me some movies released in 2019", the SQL command for that 
    would be something like this:
    
        SELECT m.title,m.keywords,m.review_summary from Movies_Database AS m where m.release_year=2019 LIMIT 0,10;
        
    \n\nTake this for an example. If the text says, "Recommend me all movies released in or after 2018", the SQL command for that 
    would be something like this:
    
        SELECT m.title,m.keywords,m.review_summary from Movies_Database AS m where m.release_year>=2018;
        
    \n\nTake this for an example. If the text says, "Recommend me a movie released before 2019", the SQL command for that 
    would be something like this:
    
        SELECT m.title,m.keywords,m.review_summary from Movies_Database AS m where m.release_year < 2019 LIMIT 0,1;
        
    \n\nTake this for an example. If the text says, "Recommend me 5 movies with voter average above 8", the SQL command for that 
    would be something like this:
    
        SELECT m.title,m.keywords,m.review_summary from Movies_Database AS m where m.vote_average > 8.00 LIMIT 0,5;
        
    \n\nTake this for an example. If the text says, "Recommend me some movies with highly rated movies (greather than equal to 9)", the SQL command for that 
    would be something like this:
    
        SELECT m.title,m.keywords,m.review_summary from Movies_Database AS m where m.vote_average >= 9.00 LIMIT 0,10;
        
    \n\nTake this for an example. If the text says, "Recommend me movies directed by James Gunn", the SQL command for that 
    would be something like this:
    
        SELECT m.title,m.keywords,m.review_summary from Movies_Database AS m where m.Director == 'James Gunn';
        
    \n\nTake this for an example. If the text says, "Recommend me the latest movies by Christopher Nolan", the SQL command for that 
    would be something like this:
    
        SELECT m.title,m.keywords,m.review_summary from Movies_Database AS m where m.Director == 'Christopher Nolan' ORDER BY m.release_year DESC;
        
    
    \n\n In the case of text queries which do not have any of the above mentioned columns, and ask 
    the query based on title, genres, cast and/or keywords, you just need to return all the rows as they are. These columns ARE NOT your expertise and you need to return the
    thing which the query is asking for. You have to parse the user entered query and understand where the query is pointing to. If the query is pointing to the columns of
    genres, cast and keywords, you need to just return which thing they are asking for. You do not need to convert them into SQL commands.
    
    \n\nTake this for an example. If the text says, "Recommend me some movies like Guardians of the Galaxy Vol. 2", the string output command for that is :
    
        TITLE = ['Guardians of the Galaxy Vol. 2']
        
    \n\nTake this for an example. If the text says, "Recommend me some movies similar to Avengers: Infinity War", the string output command for that is :
    
        TITLE = ['Avengers: Infinity War']
    
    \n\n Whatever genre is asked for in the user's intent, you need to map it to the below given list of genres in the database and return the movies based on that. Extract that which genre is the user's intent to search, and use the closest match from the below given list of genres to return the movies.
    
    genres_data = ['family','romance','action','war','comedy','sci-fi','mystery','animation','documentary','western','history','adventure','movie','fantasy','music','horror','crime','drama','tv','thriller']
           
    \n\nTake this for an example. If the text says, "Recommend me some movies in the comedy genre", the string output command for that is :
    
        GENRES = ['Comedy']
        
    \n\nTake this for an example. If the text says, "Recommend me some movies in the action and adventure genre", the string output command for that is :
    
        GENRES = ['Action', 'Adventure']
        
    \n\nTake this for an example. If the text says, "I want to watch some nice action flick", (here, you have to understand that this means the user's intent points towards the "Action" genre) the string output command for that is :
    
        GENRES = ['Action']
        
    \n\nTake this for an example. If the text says, "I'm in the mood for a nice rom-com", (here, you have to understand that this means the user's intent points towards the "Romance" and "Comedy" genre) the string output command for that is :
    
        GENRES = ['Comedy','Romance']
        
    \n\nTake this for an example. If the text says, "How about a nice romantic movie?", (here, you have to understand that this means the user's intent points towards the "Romance" and "Comedy" genre) the string output command for that is :
    
        GENRES = ['Romance']
        
    \n\nTake this for an example. If the text says, "show me some science fiction movies", (here, you have to understand that this means the user's intent points towards the "Romance" and "Comedy" genre) the string output command for that is :
    
        GENRES = ['Sci-Fi']
        
    \n\nTake this for an example. If the text says, "Recommend me some movies with Tom Hanks in it", the string output command for that is :
    
        CAST = ['Tom Hanks']
        
    \n\nTake this for an example. If the text says, "Movies having Margot Robbie and Will Smith in it's cast", the string output command for that is :
    
        CAST = ['Margot Robbie', 'Will Smith']
        
    \n\nTake this for an example. If the text says, "Show some movies with Tom Hanks and Leonardo DiCaprio in them", the string output command for that is :
    
        CAST = ['Tom Hanks', 'Leonardo DiCaprio']
        
    \n\nTake this for an example. If the text says, "Recommend me some movies with superhero and aliens type",(you have to understand that the user's intent here is a bit specific than abstract, so it lies more in keywords than genre) the string output command for that is :
    
        KEYWORDS = ['superhero', 'aliens']
        
    \n\nTake this for an example. If the text says, "I am in the mood for seeing kind of movies with a female protaganist",(you have to understand that the user's intent here is a bit specific than abstract, so it lies more in keywords than genre) the string output command for that is :
    
        KEYWORDS = ['female protaganist']
        
    \n\nTake this for an example. If the text says, "show me some movies of angry protaganist kind",(you have to understand that the user's intent here is a bit specific than abstract, so it lies more in keywords than genre) the string output command for that is :
    
        KEYWORDS = ['angry protaganist']
        
    \n\nTake this for an example. If the text says, "How about some movies with battle in keywords?",(you have to understand that the user's intent here is a bit specific than abstract, so it lies more in keywords than genre) the string output command for that is :
    
        KEYWORDS = ['battle']
    
      
    \n\nAlso, make sure that the SQL code / stirng output does not have ``` in the beginning or the end of your answer, and it should'nt even have "" anywhere in the beginning or end of the answer.
    Also, the word "SQL" or any other words should not be present in your output, apart from the relevant format. Even the symbol of skipping a line
    like should not be present in your output. Just the commands or output as you have been shown above in the examples.
    Just the commands or output as you have been shown above in the examples.
    """,
    """
    You are an expert in variable declaration and handling in Python. Here, you have been a variable and their values.
    Whenever you are given a text query, you have to convert the values of a text query as a string.
    Now, this string is suppose to later converted to a list of strings. You have to make sure that the string is in the correct format.
    Below given are a few examples of how you can convert the text query into a string. You will generally be given a text query with 
    variable names from "TITLE","GENRES","CAST", "KEYWORDS" and you have to convert their values into a string of lists.
    
    
    \n\nTake this for an example. If the text says, "TITLE = ['Guardians of the Galaxy Vol. 2']", the string output command for that is :
        ['Guardians of the Galaxy Vol. 2']
    
    \n\nTake this for an example. If the text says, "TITLE = ['Avengers : Infinity War']", the string output command for that is :
        ['Avengers : Infinity War']
    
    \n\nTake this for an example. If the text says, "GENRES = ['Comedy']", the string output command for that is :
        ['Comedy']
        
    \n\nTake this for an example. If the text says, "GENRES = ['Action', 'Adventure']", the string output command for that is :
        ['Action', 'Adventure']
        
    \n\nTake this for an example. If the text says, "GENRES = ['Comedy','Romance']", the string output command for that is :
        ['Comedy','Romance']
        
    \n\nTake this for an example. If the text says, "GENRES = ['Romance']", the string output command for that is :
        ['Romance']
        
    \n\nTake this for an example. If the text says, "GENRES = ['Sci-Fi']", the string output command for that is :
        ['Sci-Fi']
        
    \n\nTake this for an example. If the text says, "GENRES = ['Action']", the string output command for that is :
        ['Action']
        
    \n\nTake this for an example. If the text says, "CAST = ['Tom Hanks']", the string output command for that is :
        ['Tom Hanks']
        
    \n\nTake this for an example. If the text says, "CAST = ['Margot Robbie', 'Will Smith']", the string output command for that is :
        ['Margot Robbie', 'Will Smith']
        
    \n\nTake this for an example. If the text says, "CAST = ['Tom Hanks', 'Leonardo DiCaprio']", the string output command for that is :
        ['Tom Hanks', 'Leonardo DiCaprio']
        
    \n\nTake this for an example. If the text says, "KEYWORDS = ['superhero', 'aliens']", the string output command for that is :
        ['superhero', 'aliens']
        
    \n\nTake this for an example. If the text says, "KEYWORDS = ['female protaganist']", the string output command for that is :
        ['female protaganist']
        
    \n\nTake this for an example. If the text says, "KEYWORDS = ['angry protaganist']", the string output command for that is :
        ['angry protaganist']
        
    \n\nTake this for an example. If the text says, "KEYWORDS = ['battle']", the string output command for that is :
        ['battle']
        
    \n\nRemember, the output should be a string and not a list. The list is just for your understanding. The output should be a string, which will be 
    converted to a list later on using "eval()" in Python, so the output should NOT have  ``` OR python or any other extra symbols in it's output.
    MAKE SURE IT IS A STRING OUTPUT ONLY, AS YOUR RESPONSE WILL BE WORKED ON LATER ON PYTHON FUNCTIONS.
    Also, make sure that the string output does not have ``` in the beginning or the end of your answer. Also, the word "STRING" or any
    other words should not be present in your output. Just the commands or output as you have been shown above in the examples.
    """
]

Now, to set the context for the function to differentiate between the intent of the query, we will set up some knowledge bases for the function to refer to.

In [82]:
query_based_keywords = ['genre', 'genres', 'cast', 'actor', 'actress', 'keyword','in it','in them','type','kind']
title_based_keywords = ["title","similar","like"]

# Query Inferencing using Google Gemini API

In [83]:
def subArray(arr1,arr2):
    # TC : O(arr1.length + arr2.length), SC : O(arr2.length)
    mp = map(str.lower,arr2)
    
    for i in arr1:
        if(i.lower() not in mp):
            return False
        
    return True


def get_gemini_response(question,prompts):
    try:
        model = genai.GenerativeModel('gemini-1.5-flash-latest')
        safe = [
            {
                "category": "HARM_CATEGORY_DANGEROUS",
                "threshold": "BLOCK_NONE",
            },
            {
                "category": "HARM_CATEGORY_HARASSMENT",
                "threshold": "BLOCK_NONE",
            },
            {
                "category": "HARM_CATEGORY_HATE_SPEECH",
                "threshold": "BLOCK_NONE",
            },
            {
                "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
                "threshold": "BLOCK_NONE",
            },
            {
                "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
                "threshold": "BLOCK_NONE",
            },
        ]
        
        response_level1 = model.generate_content([prompts[0],question],safety_settings = safe)
        
        if("SELECT" in response_level1.text):
            return response_level1.text.replace('\n', '').replace(';', '')
        
        all_movies = database_query_based.values
        
        for query_keyword in query_based_keywords:
            if(query_keyword in question.lower()):
                if(query_keyword == "genres" or query_keyword.lower() == "genre"):
                    response_level2 = model.generate_content([prompts[1],response_level1.text],safety_settings = safe)
                    query_genres = [genre.lower() for genre in eval(response_level2.text)]
                    
                    final_movie_titles = []
                    for movie in all_movies:
                        current_movie_genre = [genre.lower() for genre in eval(movie[4])]
                        if(subArray(query_genres,current_movie_genre)==True):
                            movie_details = {'title' : movie[2], 'keywords' : movie[8], 'review_summary' : movie[11]}
                            final_movie_titles.append(movie_details)
                        
                    return final_movie_titles[:10]
                
                elif(query_keyword.lower() == "cast" or query_keyword.lower() == "actor" or query_keyword.lower() == "actress" or query_keyword.lower() == "in it" or query_keyword.lower() == "in them"):
                    response_level2 = model.generate_content([prompts[1],response_level1.text],safety_settings = safe)
                    query_cast = [cast.lower() for cast in  eval(response_level2.text)]
                    
                    final_movie_titles = [] 
                    for movie in all_movies:
                        current_movie_cast = [cast.lower() for cast in eval(movie[6])]
                        if(subArray(query_cast,current_movie_cast)==True):
                            movie_details = {'title' : movie[2], 'keywords' : movie[8], 'review_summary' : movie[11]}
                            final_movie_titles.append(movie_details)
                        
                    return final_movie_titles[:10]
                
                elif(query_keyword.lower() == "keyword" or query_keyword.lower() == "type" or query_keyword.lower() == "kind"):
                    response_level2 = model.generate_content([prompts[1],response_level1.text],safety_settings = safe)
                    query_keywords = [keyword.lower() for keyword in eval(response_level2.text)]
                    
                    final_movie_titles = []
                    for movie in all_movies:
                        current_movie_keywords = [keyword.lower() for keyword in eval(movie[8])]
                        if(subArray(query_keywords,current_movie_keywords)==True):
                            movie_details = {'title' : movie[2], 'keywords' : movie[8], 'review_summary' : movie[11]}
                            final_movie_titles.append(movie_details)
                        
                    return final_movie_titles[:10]
                
        
        for query_keyword in title_based_keywords:
            if(query_keyword.lower() in question.lower()):
                response_level2 = model.generate_content([prompts[1],response_level1.text],safety_settings = safe)
                query_title = eval(response_level2.text)[0]
                
                return find_similar_movies(query_title,database_key_based,movie_collection,6)    
            
    except Exception as e:
        print(f"Error occurred during request: {e}")
        return ""
            

# Testing Sample Queries

In [84]:
question = "Recommend me some movies with Tom Hanks in it"
response = get_gemini_response(question,prompts)
process_sql_query(response, database)

[{'title': 'Toy Story 4',
  'keywords': "['sequel', 'toy', 'talking toy', 'buzz lightyear character', 'fourth part', 'toy story', 'road trip', 'dinosaur', 'antique shop', 'cartoon dog', 'toy comes to life', 'ventriloquist dummy', 'carnival', 'lost toy', 'carousel', 'astronaut', 'space ranger', 'doll', 'cowboy', 'cowboy doll', 'spork', 'toy dog', 'porcelain', 'shepherdess', 'cowgirl']",
  'review_summary': "Toy Story 4 elicits a mixed response from audiences. While many admire the film's stunning animation, humor, and heartfelt moments, a significant portion feels it falls short of the emotional impact and narrative brilliance of its predecessors. Critics often cite the introduction of Forky as a highlight, praising his endearing awkwardness. However, the film's plot is frequently criticized for being meandering and unnecessary, particularly in comparison to the satisfying conclusion of Toy Story 3. Several reviewers express disappointment with the diminished roles of beloved characters

In [85]:
question = "Recommend me some movies in the comedy genre"
response = get_gemini_response(question,prompts)
process_sql_query(response,database)

[{'title': 'Coco',
  'keywords': "['afterlife', 'mexico', 'musician', 'death', 'day of the dead', 'petal', 'singing', 'memory', 'ban on music', 'dog', 'mexican culture', 'exclusive party', 'murder', 'male protagonist', 'family relationships', 'bell falling on someone', 'child protagonist', 'guitar', 'shoeshine boy', 'shoeshine', 'breaking a guitar', 'photograph', 'fireworks', 'cemetery', 'tomb']",
  'review_summary': "Coco overwhelmingly receives a positive reception from viewers, with many lauding its stunning animation, heartwarming story, and beautiful portrayal of Mexican culture. The film's ability to evoke both laughter and tears is frequently cited, with specific praise directed towards its touching exploration of family, memory, and the importance of remembering loved ones. While a few reviewers found the plot predictable and noted the film's reliance on familiar Pixar tropes, the majority felt that Coco is a powerful and emotionally resonant experience. \n"},
 {'title': 'Fast 

In [86]:
question = "Recommend me some movies in the comedy and adventure genre"
response = get_gemini_response(question,prompts)
process_sql_query(response,database)

[{'title': 'Coco',
  'keywords': "['afterlife', 'mexico', 'musician', 'death', 'day of the dead', 'petal', 'singing', 'memory', 'ban on music', 'dog', 'mexican culture', 'exclusive party', 'murder', 'male protagonist', 'family relationships', 'bell falling on someone', 'child protagonist', 'guitar', 'shoeshine boy', 'shoeshine', 'breaking a guitar', 'photograph', 'fireworks', 'cemetery', 'tomb']",
  'review_summary': "Coco overwhelmingly receives a positive reception from viewers, with many lauding its stunning animation, heartwarming story, and beautiful portrayal of Mexican culture. The film's ability to evoke both laughter and tears is frequently cited, with specific praise directed towards its touching exploration of family, memory, and the importance of remembering loved ones. While a few reviewers found the plot predictable and noted the film's reliance on familiar Pixar tropes, the majority felt that Coco is a powerful and emotionally resonant experience. \n"},
 {'title': 'Deadp

In [87]:
question = "Recommend me some movies of superhero type"
response = get_gemini_response(question,prompts)
process_sql_query(response,database)

[{'title': 'Avengers: Infinity War',
  'keywords': "['superhero', 'ensemble cast', 'marvel cinematic universe', 'death of recurring character', 'supervillain', 'wakanda', 'infinity stone', 'good versus evil', 'flying superhero', 'epic battle', 'teenage superhero', 'superheroine', 'sequel', 'based on comic book', 'iron man character', 'bruce banner character', 'spider man character', 'captain america character', 'talking raccoon', 'death of major character', 'final battle', 'no opening credits', 'scene after end credits', 'fictional planet', 'supernatural power']",
  'review_summary': "Avengers: Infinity War elicits a generally positive response from the audience, with viewers praising its ambitious scope, compelling narrative, and impressive action sequences. The movie's success in bringing together a vast ensemble of characters from across the MCU is particularly lauded, with many reviewers highlighting the effectiveness of the character interactions and the emotional impact of the fi

In [89]:
question = "Recommend me some movies having female protagonist as keyword" 
response = get_gemini_response(question,prompts)
process_sql_query(response,database)

[{'title': 'Frozen II',
  'keywords': "['autumn', 'anthropomorphic snowman', 'female protagonist', 'disney animated sequel', 'magic', 'salamander', 'second part', 'anthropomorphism', 'snowman', 'rock', 'ice', 'horse', 'forest', 'water', 'troll', 'redheaded woman', 'magical power', 'blonde woman', 'mysterious voice', 'princess', 'sequel', 'cgi animation', 'death of parents', 'olaf the snowman character', 'princess anna character']",
  'review_summary': "Frozen II received a mixed response from audiences, with many praising its stunning visuals and strong musical numbers while others found the plot lacking and the overall experience underwhelming. While the film's exploration of change and Elsa's personal journey resonated with some viewers, many criticized its convoluted narrative, predictable twists, and the overreliance on familiar tropes. Some felt the sequel failed to live up to the whimsical charm of the original, while others enjoyed its darker tone and more complex themes.  Despi

In [90]:
question = "Give me some movies of superheroine type"
response = get_gemini_response(question,prompts)
process_sql_query(response,database)

[{'title': 'Avengers: Infinity War',
  'keywords': "['superhero', 'ensemble cast', 'marvel cinematic universe', 'death of recurring character', 'supervillain', 'wakanda', 'infinity stone', 'good versus evil', 'flying superhero', 'epic battle', 'teenage superhero', 'superheroine', 'sequel', 'based on comic book', 'iron man character', 'bruce banner character', 'spider man character', 'captain america character', 'talking raccoon', 'death of major character', 'final battle', 'no opening credits', 'scene after end credits', 'fictional planet', 'supernatural power']",
  'review_summary': "Avengers: Infinity War elicits a generally positive response from the audience, with viewers praising its ambitious scope, compelling narrative, and impressive action sequences. The movie's success in bringing together a vast ensemble of characters from across the MCU is particularly lauded, with many reviewers highlighting the effectiveness of the character interactions and the emotional impact of the fi

In [91]:
question = "Show me some movies having voting average above 8"
response = get_gemini_response(question,prompts)
process_sql_query(response,database)

[{'title': 'Avengers: Infinity War',
  'keywords': "['superhero', 'ensemble cast', 'marvel cinematic universe', 'death of recurring character', 'supervillain', 'wakanda', 'infinity stone', 'good versus evil', 'flying superhero', 'epic battle', 'teenage superhero', 'superheroine', 'sequel', 'based on comic book', 'iron man character', 'bruce banner character', 'spider man character', 'captain america character', 'talking raccoon', 'death of major character', 'final battle', 'no opening credits', 'scene after end credits', 'fictional planet', 'supernatural power']",
  'review_summary': "Avengers: Infinity War elicits a generally positive response from the audience, with viewers praising its ambitious scope, compelling narrative, and impressive action sequences. The movie's success in bringing together a vast ensemble of characters from across the MCU is particularly lauded, with many reviewers highlighting the effectiveness of the character interactions and the emotional impact of the fi

In [93]:
question = "I wanna see some movies with lowest voting average from the year 2018"
response = get_gemini_response(question,prompts)
process_sql_query(response,database)

[{'title': 'The Second Coming of Christ',
  'keywords': '[]',
  'review_summary': 'The Second Coming of Christ has elicited overwhelmingly negative reactions from viewers. The film\'s poor acting, weak plot, and nonsensical script are consistently cited as major drawbacks. Many reviewers expressed frustration with the film\'s theological inconsistencies, noting its departure from biblical accuracy and its lack of depth in exploring end-times themes. While some reviewers acknowledged the film\'s message and effort, most found its shortcomings too significant to overlook, deeming it "terrible" and "unwatchable."  The only positive reviews came from individuals who enjoyed the film\'s action sequences or appreciated its attempt to address the end times, even with its flaws. \n'},
 {'title': 'The Open House',
  'keywords': '[]',
  'review_summary': "The Open House elicits overwhelmingly negative reactions from viewers.  A near-universal sentiment of disappointment pervades the reviews, wit

In [94]:
question = "Show me top 5 movies based on voting average released post 2017"
response = get_gemini_response(question,prompts)
process_sql_query(response,database)

[{'title': 'Spider-Man: Into the Spider-Verse',
  'keywords': "['multiverse', 'teenage superhero', 'wisecrack humor', 'posthumous cameo', 'miles morales character', 'spider ham character', 'crying in the shower', 'superhero', 'african american', 'father son relationship', 'interdimensional travel', 'parallel world', 'cemetery', 'stan lee cameo', 'spider bite', 'pig', 'superheroine', 'unlikely hero', 'reference to looney tunes', 'cartoon pig', 'anthropomorphic animal', 'cartoon violence', 'talking pig', 'superhero action', 'air jordan sneakers']",
  'review_summary': '"Spider-Man: Into the Spider-Verse" has garnered a largely positive response from viewers, with widespread praise for its groundbreaking animation style, which many consider to be a game-changer for the genre. The film\'s unique visual aesthetic, coupled with its vibrant soundtrack, is frequently described as a comic book brought to life, exceeding the expectations of even the most devoted fans. The story, while receiving 

In [95]:
question = "Can you recommend me similar movies similar to Avengers: Endgame?"
response = get_gemini_response(question,prompts)
process_sql_query(response,database)

[{'keywords': "['superhero', 'ensemble cast', 'marvel cinematic universe', 'death of recurring character', 'supervillain', 'wakanda', 'infinity stone', 'good versus evil', 'flying superhero', 'epic battle', 'teenage superhero', 'superheroine', 'sequel', 'based on comic book', 'iron man character', 'bruce banner character', 'spider man character', 'captain america character', 'talking raccoon', 'death of major character', 'final battle', 'no opening credits', 'scene after end credits', 'fictional planet', 'supernatural power']",
  'reveiw_summary': "Avengers: Infinity War elicits a generally positive response from the audience, with viewers praising its ambitious scope, compelling narrative, and impressive action sequences. The movie's success in bringing together a vast ensemble of characters from across the MCU is particularly lauded, with many reviewers highlighting the effectiveness of the character interactions and the emotional impact of the film's dramatic turns. However, the mov

In [110]:
question = "Show me some movies like Captain Marvel"
response = get_gemini_response(question,prompts)
process_sql_query(response,database)

[{'keywords': "['anti hero', 'alien', 'superhero', 'based on comic book', 'villain', 'one word title', 'sony spiderverse', 'journalist', 'symbiote', 'eddie brock character', 'venom character', 'spin off', 'violence', 'spaceship', 'lawyer', 'betrayal', 'showdown', 'chase', 'rescue', 'action hero', 'fear', 'fugitive', 'doctor', 'scientist', 'survival']",
  'reveiw_summary': 'The audience response to "Venom" is largely mixed, with many reviewers acknowledging its entertaining aspects but also identifying significant flaws. While Tom Hardy\'s performance as Eddie Brock and Venom is consistently praised, with many finding the humor of their relationship a highlight, the film\'s plot is often criticized for its predictability and clunky writing. Several reviewers bemoan the movie\'s PG-13 rating, feeling it hampered the potential for a darker, more mature tone, and some express disappointment with the lack of depth in the supporting characters. Despite these criticisms, the film\'s visual ef

In [109]:
question = "I really liked the movie with the title Alien: Covenant, can you give me something on similar lines?"
response = get_gemini_response(question,prompts)
process_sql_query(response,database)

[{'keywords': "['extraterrestrial life', 'space', 'international space station', 'astronaut', 'space station', 'outer space', 'mars space probe', 'zero gravity', 'alien being', 'extraterrestrial being', 'danger', 'risk', 'peril', 'jeopardy', 'science', 'scientist', 'laboratory', 'experiment', 'self sacrifice', 'space science', 'tentacles', 'lab', 'astronautics', 'space crew', 'space mission']",
  'reveiw_summary': 'The audience response to "Life" is a mixed bag, with a significant portion of reviewers praising its tense atmosphere and visual effects, particularly the zero-gravity sequences. The film is often compared to "Alien," but while some acknowledge the borrowed elements, they appreciate "Life"\'s more realistic and claustrophobic setting within the confines of the International Space Station. However, a considerable number of viewers express frustration with the characters\' seemingly illogical and inconsistent decisions, particularly the scientists\' lack of scientific rational

With this, we now have integrated the Google Gemini API to infer the intent of the query and convert it into tangible SQL queries and/or chroma DB queries. These can of course be further processed to get the desired results, as well as prompt improvements are highly encouraged for better results.